In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# torch.manual_seed(0)

from torch.autograd import Variable

#from torchsummary import summary

import torch.optim as optim
import time
import matplotlib
from torch.utils.data import DataLoader
import torch.utils.data as data
import numpy as np
import pandas as pd
from torchvision.datasets import ImageFolder
from torchvision import transforms
import py_func.Deep_learning_torch_func as DL
import py_func.models_func as my_model
import py_func.dat_rb_func as drb
import cv2

In [2]:
from torchinfo import summary

In [ ]:
simpath = '../dat/simdat/pjr/1byte/random/m_500_0.2_2_sim.dat'
binpath = '../dat/bindat/1byte/m_28_1.dat'
ori_sizex = 28
ori_sizey = 28

#復元
m_dc_set_t,m_dc_set_v=DL.mysimbin_to_dataset(simpath,binpath,SX,ori_sizex,ori_sizey,N_m,1,1,True,60000)
m_dc_loader_t=DataLoader(m_dc_set_t,batch_size=64,shuffle=True)
m_dc_loader_v=DataLoader(m_dc_set_v,batch_size=64,shuffle=True)